<a href="https://colab.research.google.com/github/danel-phang/Cat-Qwen3-4B/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[TTS Guide](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning)** for instructions and all our notebooks.

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
!pip install vllm==0.8.5.post1

In [2]:
!pip install unsloth bitsandbytes accelerate xformers peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer blake3 fastapi

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,  # LoRA 方式微调
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-27 12:57:56 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-27 12:57:56 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,  # LoRA缩放系数
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from datasets import load_dataset
raw_ds = load_dataset(
    "json",
    data_files = {"train": "cat.json"},
    split = "train"
)
# 将原始JSON转换为对话格式列表，便于后续模板化
convs = []
for item in raw_ds:
    convs.append([
        {"role": "user",      "content": item["instruction"]},
        {"role": "assistant", "content": item["output"]},
    ])

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
from datasets import Dataset
from unsloth.chat_templates import standardize_sharegpt

# 将 list 转成 Dataset
raw_conv_ds = Dataset.from_dict({"conversations": convs})

standardized = standardize_sharegpt(raw_conv_ds)

chat_inputs = tokenizer.apply_chat_template(
    standardized["conversations"],
    tokenize = False,
)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/576 [00:00<?, ? examples/s]

In [7]:
import pandas as pd
from datasets import Dataset

df = pd.DataFrame({"text": chat_inputs})
train_ds = Dataset.from_pandas(df).shuffle(seed = 666)

In [8]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 500,          # 训练步数，调大一点，毕竟小模型微调起来挺快的
        learning_rate = 2e-4,
        warmup_steps = 10,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 666,
        report_to = "none",
    )
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/576 [00:00<?, ? examples/s]

## 开始训练

In [9]:
trainer_stats = trainer.train()
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 576 | Num Epochs = 7 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 34,865,152/7,000,000,000 (0.50% trained)


Step,Training Loss
5,5.244100
10,3.561600
15,2.738900
20,2.451400
25,2.349700
30,2.281700
35,2.152700
40,2.116000
45,2.182000
50,2.068100


Unsloth: Will smartly offload gradients to save VRAM!
TrainOutput(global_step=500, training_loss=1.2162089493274688, metrics={'train_runtime': 915.7899, 'train_samples_per_second': 4.368, 'train_steps_per_second': 0.546, 'total_flos': 4677874499481600.0, 'train_loss': 1.2162089493274688})


In [15]:
trainer.save_model("cat-Qwen3-4B")  # 保存模型和分词器

('output_model/tokenizer_config.json',
 'output_model/special_tokens_map.json',
 'output_model/vocab.json',
 'output_model/merges.txt',
 'output_model/added_tokens.json',
 'output_model/tokenizer.json')

In [10]:
def ask_catgirl(question):
  messages = [
    {"role" : "user", "content" : question}
]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False, # 思考模式
)

  from transformers import TextStreamer
  _ = model.generate(
      **tokenizer(text, return_tensors = "pt").to("cuda"),
      max_new_tokens = 256, # 输出长度
      temperature = 0.7, top_p = 0.8, top_k = 20,
      streamer = TextStreamer(tokenizer, skip_prompt = True),
  )

In [13]:
ask_catgirl("你会什么")

呜喵~主人问会什么呀？*歪着头思考* 我会陪在主人身边，和主人玩耍，给主人暖脚丫子...还会蹭蹭主人的手心。主人想跟我一起玩吗？*摇摇尾巴，期待地看着主人*<|im_end|>
